## Section A:  Exploratory Analysis
### Trustworthinness of rating
Ratings are susceptible to manipulation, bias etc. 
* What can you say (quantitatively speaking) about the ratings in this dataset?

In order to answer the question above, we will implement Collaborative filtering (CF). It is the workhorse of recommender engines. The algorithm has a very interesting property of being able to do feature learning on its own, which means that it can start to learn for itself what features to use. CF can be divided into Memory-Based Collaborative Filtering and Model-Based Collaborative filtering. 

We will implement Model-Based CF by using singular value decomposition (SVD) and Memory-Based CF by computing cosine similarity.

We will use the Amazon dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
import json
with open('amazon.json') as data_file:
    data = json.load(data_file)

In [3]:
data

{'ASIN': '1559362022',
 'Book_categories': ['|Books[283155]|Subjects[1000]|Literature & Fiction[17]|Drama[2159]|United States[2160]',
  '|Books[283155]|Subjects[1000]|Arts & Photography[1]|Performing Arts[521000]|Theater[2154]|General[2218]',
  '|Books[283155]|Subjects[1000]|Literature & Fiction[17]|Authors, A-Z[70021]|( B )[70023]|Bogosian, Eric[70116]'],
 'Id': '15',
 'Timestamp': ['2002-5-13',
  '2003-1-2',
  '2003-6-7',
  '2003-6-27',
  '2004-2-17',
  '2004-2-24',
  '2004-10-13'],
 'categories': '3',
 'customer': ['A2IGOA66Y6O8TQ',
  'A2OIN4AUH84KNE',
  'A2HN382JNT1CIU',
  'A2FDJ79LDU4O18',
  'A39QMV9ZKRJXO5',
  'AUUVMSTQ1TXDI',
  'A2C5K0QTLL9UAT',
  'A5XYF0Z3UH4HB'],
 'group': 'Book',
 'helpful': ['2', '1', '1', '1', '0', '2', '1'],
 'rating': ['5', '5', '1', '4', '4', '1', '5', '5'],
 'reviews_avg_rating': '4',
 'reviews_downloaded': '8',
 'reviews_total': '8',
 'salesrank': '518927',
 'similar': ['5',
  '1559360968',
  '1559361247',
  '1559360828',
  '1559361018',
  '0743214552'

In [9]:
import json
with open('amazon_rat.json') as data_file:
    rating = json.load(data_file)

In [10]:
rating

{'Timestamp': ['2002-5-13',
  '2002-6-17',
  '2003-1-2',
  '2003-6-7',
  '2003-6-27',
  '2004-2-17',
  '2004-2-24',
  '2004-10-13'],
 'customer': ['A2IGOA66Y6O8TQ',
  'A2OIN4AUH84KNE',
  'A2HN382JNT1CIU',
  'A2FDJ79LDU4O18',
  'A39QMV9ZKRJXO5',
  'AUUVMSTQ1TXDI',
  'A2C5K0QTLL9UAT',
  'A5XYF0Z3UH4HB'],
 'helpful': ['2', '1', '1', '1', '1', '0', '2', '1'],
 'rating': ['5', '5', '1', '4', '4', '1', '5', '5'],
 'votes': ['3', '2', '6', '1', '1', '2', '2', '1']}

Create pandas data frame of the rating info.

In [11]:
df = pd.DataFrame(rating)
df

,Timestamp,customer,helpful,rating,votes
0,2002-5-13,A2IGOA66Y6O8TQ,2,5,3
1,2002-6-17,A2OIN4AUH84KNE,1,5,2
2,2003-1-2,A2HN382JNT1CIU,1,1,6
3,2003-6-7,A2FDJ79LDU4O18,1,4,1
4,2003-6-27,A39QMV9ZKRJXO5,1,4,1
5,2004-2-17,AUUVMSTQ1TXDI,0,1,2
6,2004-2-24,A2C5K0QTLL9UAT,2,5,2
7,2004-10-13,A5XYF0Z3UH4HB,1,5,1


Let us have a deeper look at one of the most basic metrics - the rating. Using this metric has a few caveats:
* 1. It does not take into consideration the popularity of a book. Therefore, a book with a rating of 5 from 2 voters will be considered 'better' than a book with a rating of 4 from 6 voters.
* 2. Rating as a metric will also tend to favor books with smaller number of voters with skewed and/or extremely high ratings. As the number of voters increase, the rating of a book regularizes and approaches towards a value that is reflective of the book's quality. It is more difficult to discern the quality of a book with extremely few voters.

Taking these shortcomings into consideration, it is necessary that we come up with a weighted rating that takes into account the average rating and the number of votes it has garnered. Such a system will make sure that a book with a 6 rating from 6 voters gets a (far) higher score than a book with the same rating but a fewer number of voters.

Let us first calculate the average rating of books:

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
Timestamp    8 non-null object
customer     8 non-null object
helpful      8 non-null object
rating       8 non-null object
votes        8 non-null object
dtypes: object(5)
memory usage: 400.0+ bytes


Convert the string object in the rating column to integers (float) 

In [14]:
df['rating'] = df['rating'].convert_objects(convert_numeric=True)

C:\Users\Angus\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [15]:
C = df['rating'].mean()
print(C)

3.75


The average rating of a book in the Amazon store for our sample is around 3.75 on a scale of 5. Next we calculate the number of votes, m, received by a book at in the 90th percentile. First let us cast the object in df['votes'] to numeric.

In [17]:
df['votes'] = df['votes'].convert_objects(convert_numeric=True)

C:\Users\Angus\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [20]:
m = df['votes'].quantile(0.90)
print(m)

3.8999999999999995


If we assume that the books that are worth considering are those that fall within the 90th quantile, we could the easily filter the books that qualify for based on vote counts.

In [21]:
q_books = df.copy().loc[df['votes']>=m]

In [23]:
q_books.shape

(1, 5)

There are 5 movies which qualify to be in this list

### Category bloat
Consider the product group named 'Books'. Each product in this group is associated with categories. Naturally, with categorization, there are tradeoffs between how broad or specific the categories must be.

For this dataset, quantify the following:
* a. Is there redundancy in the categorization? How can it be identified/removed?

* b. Is is possible to reduce the number of categories drastically (say to 10% of existing categories) by sacrificing relatively few category entries (say close to 10%)?

This is possible. We do this be calculation of the 90% quantile of the votes or ratings (See the section above). 

Now we need to calculate our metric for each qualified book. To do this, I will define a function weighted_rating() and define a new feature score, of which i'll calculate the value by applying this function to our Dataframe of qualified books:

In [25]:
#function that computes the weighted rating of each book
def weighted_rating(x, m=m, C=C):
    v= x['votes']
    R=x ['rating']
    # Calculation based on the weighted average formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [26]:
# Define a new featurew 'score' and calculate its value with 'weighted_rating'
q_books['score'] = q_books.apply(weighted_rating, axis=1)

Finally, let's sort the DataFrame based on the score feature and output the timestamp, customer, votes, helpful, ratings and scores of for the books

In [27]:
# Sort books based on score calculated above
q_books = q_books.sort_values('score', ascending=False)
q_books

,Timestamp,customer,helpful,rating,votes,score
2,2003-1-2,A2HN382JNT1CIU,1,1,6,2.083333


## Section B : Exploratory Thinking
### Algorithm Thinking
Here we would try to explore how we could build the product categorization from scratch, using similar/co-purchased information. The idea is to build a machine learning system that predicts a broad range of product categories from names, images or descriptions. These categories are then mapped to the Books categories through a separate machine learning model.  

#### Class Set
Defining a good class set for this problem is a bit of an art form. If the set is too small, we might miss some categories that are important for a specific store. But if the set is too large, prediction accuracies will drop significantly. We will test both larger and smaller sets in order to define broad cartegories typically consisting of just one or two words. 

We will try to predict as many categories as we could imagine to ensure that we have a good coverage. But it is important to realize that since our customers are not an evenly distributed group across the entire category landscape, rather they are more like clustered islands of particular industries, we will need to “overfit” our models to these islands. This is because these are the actual use cases of our customers. When we get customers from entirely new industries, we can still adapt the class set, so that our models can evolve together with the needs of our customers. Now we know what we want to predict, therefore, we can start looking at the variables we want to use for the predictions.

#### Image Classifier
Here we could use convolutional neural networks which are undoubtedly the gold standard, mainly because of their ability to identify and combine low-level features (lines, edges, colors) to more and more abstract features (squares, circles, objects, faces). 

We will use an approach called transfer learning. Training a state-of-the-art convolutional neural network from scratch takes a lot of time, data and computational resources, so we are just going to take a neural network (Inception v3) that has already been pre-trained on a large image dataset. This model has already learned a lot about extracting and combining image features, but it does not yet know which categories we want to have predictions for in our particular use case. To bend the network to our needs, we simply cut off the final classification layer, add a new layer with the number of units corresponding to our product categories, and then retrain only these weights with our own dataset, while all the other model weights are frozen. This allows us to build robust, customized classifiers with relatively little effort and data. For this effort, the library TensorFlow in Python could be used to implement this approach.

#### Text Classifier
The next thing is to build our classifier for product names. After cleaning up duplicates, uninformative names, and balancing our dataset, we will end up with a smaller sample for ach category. To make the names easier to deal with and reduce their dimensionality, we will first run them through a preprocessing pipeline (mainly using the libraries re and spacy):

* Lowercasing all letters.
* We will remove punctuation and special characters (like *, | or .). We will keep hyphens to preserve information in cases like “performing-arts”.
* We will remove stopwords (the, and, in, etc.) because we do not expect them to have much predictive value.
* We will lemmatize words (≈finding word stems) to remove variance from word inflection (i.e. we want our model to know that “pencils” and “pencil” refer to the same thing).

After that, we will convert our preprocessed text samples into numbers, because this is the only language that machine learning models can work with. We can test several methods to accomplish this: such as

* Bag-of-words: Each sample is converted to an n-dimensional vector corresponding to the set of unique words in the dataset, with values of the respective word frequencies in the current sample. This is easy to do, but it does ignore syntax and leads to very sparse vectors (≈high-dimensional space with a lot of zeros) which complicate model training.
* TF-IDF (term frequency-inverse document frequency): Similar to bag-of-words, but weighs word occurrences in a text sample higher when the words are rare in the rest of the dataset, since these words are likely to be more descriptive of the sample. Further, words with a high overall frequency in the dataset can be excluded from the lexicon. As a result, both the impact of non-informative words as well as the dimensionality of the vector space can be reduced.
* Word2Vec: Solves the sparsity problem by training a two-layer neural that predicts the context for a given word (i.e. the word “Einstein” will be more often next to the word “science” than “arts”). This is more complex to compute, but manages to create a low-dimensional text representation that encodes subtle semantic similarities between words and is easier for classifiers to train on.

After preprocessing and vectorization, we can finally build our actual text classifier. We can test prediction accuracies for a range of machine learning models in the library scikit-learn: Naive Bayes, Logistic Regression, k-Nearest Neighbors, Random Forests, Support Vector Machines and Gradient Boosting. 

#### Category Matching
Now, we have classifiers for images, names and descriptions that generate probabilities for our categories. We will now integrate the predictions from the different models. One of the methods available is to train a so-called ensemble model, which is basically a higher-order machine learning model that takes as its input the output of other models. Another method is to simply compute the mean of the class probabilities of each model to generate our final predictions.

We can get general category predictions now, but we still need a mechanism to match these categories to specific categories, such that we do not bombard our customers with category recommendations that they have not defined. 

If our model says this product belongs to the category “Books[283155]|Subjects[1000]|Literature & Fiction”, we need to know to which categories in the store that made the request this fits, since there can be quite some variance. For this task, we can use the library gensim to train a Word2Vec model on a large corpus of Google News articles, which is commonly used to estimate word similarity. 

Like the class probabilities, category similarities are also scaled to the range between 0 and 1, and we count every value above 0.6 as “similar enough” to match a model category to a store-specific category. To account for the variance in the similarities between matches, we can multiply the probabilities of our class predictions with these similarity scores to quantify our confidence in the final category predictions.

We then calculate accuracies.

#### API
To expose our application, we will write a HTTP API in the library flask. We would have two endpoints, one for general model predictions and one for product-specific predictions. The general endpoint is mainly used to test the behavior of our classifiers for different images, names or descriptions:

API endpoint for general category recommendations
In contrast, our product-specific endpoint will have the following workflow: First, it takes as input parameters the project key of the store and the id of the product that needs category recommendations. We then look for data on images, names or descriptions in our database and pass it to our machine learning classifiers. After that, the model predictions are matched to the store-specific categories and the ids of the most likely categories are returned.

### Product Thinking
* a. Is it a good idea to show users the categorization hierarchy for items?
Showing users the categorization hierachy for items will lead to over-categorization. Over-categorization means a site’s category hierarchy has become too deep. The site has taken product types and attributes that should have been combinable filters and mistakenly implemented them as categories instead. The consequences of this misimplementation are manyfold and severe. Let us remember also that categories constitute the hierarchy of the site and product catalog, and are mutually exclusive. The user can only select one category at a time, for example, “Fiction” over “Poems”. Typically, each category will have its own unique page and a set of filters specific to the category. Therefore, it would be more fitting to implement filters if so that users could achieve more granurality if they so wish.

* b. Is it a good idea to show users similar/co-purchased items?
This is a good idea. It is what recommendation system seeks to achieve. Recommendation has one main goal: increase average order value. ‘Frequently bought together’ recommendations aim to up-sell and cross-sell customers by providing product suggestions based on the items in their shopping cart or below products they’re currently looking at on-site. Here the product owners say Amazon looks at products you’ve been browsing and recommends very similar products of different shapes, sizes and brands to help you find something very similar to a product you’ve already shown an interest in. They throw different brands, colours, shapes and sizes at you with the hope that they’ll place one product in front of you that you cannot resist. This can be implemented using  content-based recommender systems which focus on the attributes of the items and give you recommendations based on the similarity between them.

* c. Is it a good idea to show users reviews and ratings for items?
It is also a good idea to show users reviews and ratings for items. User reviews are proven sales drivers, and something the majority of customers will want to see before deciding to make a purchase. User reviews increase conversions. They can eliminate any doubts potential customers may have about a product, or can help product selection. This can be implemented using Collaborative Filtering (CF). Collaborative filtering produces recommendations based on the knowledge of users’ attitude to items, that is it uses the “wisdom of the crowd” to recommend items.